In [ ]:
N = 2 ** 4
M = N // 4

# Generate a deterministic DBBD matrix
A = generate_DBDD_matrix(M, N)
print('DBDD:')
printFormatted(A)

# Generate a random Gaussian matrix
B = generate_random_matrix(M, N, matrix_type='gaussian')
print('Gaussian:')
printFormatted(B)


# Generate a random scaled binary matrix
C = generate_random_matrix(M, N, matrix_type='scaled_binary')
print('Scaled binary:')
printFormatted(C)

# Generate a random unscaled binary matrix
D = generate_random_matrix(M, N, matrix_type='unscaled_binary')
print('Unscaled binary:')
printFormatted(D)


#### Generate dictionary

In [ ]:
DIM = 16

# Generate dictionary
A = generate_dct_dictionary(DIM)  # test what you want
print(f'Dictionary shape: {A.shape}')
print()

# Orthonormality test
print("Orthonormality test")

# Check orthogonality
rankA = np.linalg.matrix_rank(A)
#print(f'Rank of A: {rankA}')
if A.shape[0] == rankA or A.shape[1] == rankA:
    print("Matrix A is full rank.")

# Check normalization
is_normalized = check_normalization(A)
print(f"Matrix A is normalized: {is_normalized}")
is_normalized = check_normalization(A.T)
print(f"Matrix A^T is normalized: {is_normalized}")
print()

# Coherence test
print("Coherence test")
coherence = compute_coherence(A)
print(f'Coherence: {coherence}')
